In [1]:
import sys

sys.path.append('/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/lib/')
sys.path.append('/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/cffi_interface/')

In [2]:
import cffi_plugin

We import the cffi_plugin like a Python library and try to execute a function following the description in 

https://cffi.readthedocs.io/en/latest/embedding.html#issues-about-using-the-so --> testing

In [3]:
cffi_plugin.__dict__

{'__name__': 'cffi_plugin',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader at 0x7ffff1050d00>,
 '__spec__': ModuleSpec(name='cffi_plugin', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x7ffff1050d00>, origin='/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/lib/cffi_plugin.so'),
 'ffi': <_cffi_backend.FFI at 0x7ffff08094a0>,
 'lib': <Lib object for 'cffi_plugin'>,
 '__file__': '/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/lib/cffi_plugin.so'}

In [4]:
from cffi_plugin import ffi, lib

In [5]:
lib.__dict__

{'i_add_emi_echam_ttr': <cdata 'void(*)(int *, int *, int *, int *, double *, double *, double *, double *, double *, double *)' 0x7ffff07e0310>,
 'i_add_one': <cdata 'void(*)(int *, double *)' 0x7ffff07e0370>,
 'i_check_interface': <cdata 'void(*)()' 0x7ffff07e03a0>,
 'i_get_emi_float': <cdata 'void(*)(double *)' 0x7ffff07e03c0>,
 'i_get_emi_number': <cdata 'void(*)(int *)' 0x7ffff07e03e0>,
 'i_hello_world': <cdata 'void(*)()' 0x7ffff07e0400>,
 'i_warm_rain_nn': <cdata 'void(*)(int *, int *, int *, int *, double *, double *, double *, double *, double *, double *, double *, double *)' 0x7ffff07e0420>}

In [6]:
lib.i_hello_world

<cdata 'void(*)()' 0x7ffff07e0400>

In [7]:
lib.i_hello_world()

/home/k/k202141/.conda/envs/iconml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello from the Python World!


In [15]:
import transfer_arrays

In [17]:
transfer_arrays.asarray(ffi, '0',  shape=(5,1))

error: identifier expected
0
^

In [20]:
shape=(4,0)
import numpy as np
np.product(shape)

0

In [23]:
import os

In [24]:
ff = '/home/k/k202141/jobdir/hereon_iconml/LOG_fortran_reference_with_moments'
cf = '/home/k/k202141/jobdir/hereon_iconml/LOG_cffi_with_moments'

In [56]:
def get_first_moments(logfile):
    with open(logfile) as f:
        line = f.readline()
        in_first_timestep = False
        
        non_zero_moments = dict(n_cloud_t0=[],
                               q_cloud_t0=[],
                               n_rain_t0=[],
                               q_rain_t0=[],
                               n_cloud_t1=[],
                               q_cloud_t1=[],
                               n_rain_t1=[],
                               q_rain_t1=[])
    
        
        while line:
            if '_t0 :' in line or '_t1 :' in line:
                splits = line.split(':')
                key = splits[1].split()[1]
                val = float(splits[2])
                in_first_timestep = True
                
                non_zero_moments[key].append(val)
            elif in_first_timestep:
                print(line)
                break
            
            line = f.readline()
            
        return non_zero_moments

In [90]:
def get_first_new_moments(logfile):
    with open(logfile) as f:
        line = f.readline()
        in_first_timestep = False
        
        non_zero_moments = dict(n_cloud_t0=[],
                               q_cloud_t0=[],
                               n_rain_t0=[],
                               q_rain_t0=[],
                               n_cloud_t1=[],
                               q_cloud_t1=[],
                               n_rain_t1=[],
                               q_rain_t1=[])
    
        
        while line:
            if '_t1 :' in line:
                splits = line.split(':')
                key = splits[1].split()[1]
                val = float(splits[2])
                in_first_timestep = True
                
                non_zero_moments[key].append(val)
            elif '_t0 :' in line:
                pass
            elif in_first_timestep:
                print(line)
                break
            
            line = f.readline()
            
        return non_zero_moments

In [91]:
fmom = get_first_moments(ff)
cmom = get_first_moments(cf)
fmom1 = get_first_new_moments(ff)
cmom1 = get_first_new_moments(cf)

0:  mo_2mom_mcrph_driver: two moment mcrph ends!

0:  mo_2mom_mcrph_driver: two moment mcrph ends!

0:  mo_2mom_mcrph_driver: two moment mcrph ends!

0:  mo_2mom_mcrph_driver: two moment mcrph ends!



In [67]:
from matplotlib import pyplot as plt
%matplotlib inline

In [73]:
for key in fmom:
    if 't0' in key:
        assert np.array(fmom[key] == cmom[key])

In [74]:
for key in fmom:
    if 't1' in key:
        print(fmom[key])
        print(cmom[key])

[14836000.0, 14836000.0, 14836000.0, 14836000.0, 148760000.0, 148760000.0, 148760000.0, 148760000.0, 150270000.0, 150270000.0, 150270000.0, 150270000.0]
[]
[7.7681e-06, 7.7681e-06, 7.7681e-06, 7.7681e-06, 4.7928e-05, 4.7928e-05, 4.7928e-05, 4.7928e-05, 3.3131e-05, 3.3131e-05, 3.3131e-05, 3.3131e-05]
[]
[9.0825e-06, 9.0825e-06, 9.0825e-06, 9.0825e-06, 0.00013062, 0.00013062, 0.00013062, 0.00013062, 2.9176e-05, 2.9176e-05, 2.9176e-05, 2.9176e-05]
[]
[2.3614e-15, 2.3614e-15, 2.3614e-15, 2.3614e-15, 3.3962e-14, 3.3962e-14, 3.3962e-14, 3.3962e-14, 7.5857e-15, 7.5857e-15, 7.5857e-15, 7.5857e-15]
[]


In [76]:
cmom

{'n_cloud_t0': [14836000.0,
  14836000.0,
  14836000.0,
  14836000.0,
  148760000.0,
  148760000.0,
  148760000.0,
  148760000.0,
  150270000.0,
  150270000.0,
  150270000.0,
  150270000.0],
 'q_cloud_t0': [7.7681e-06,
  7.7681e-06,
  7.7681e-06,
  7.7681e-06,
  4.7928e-05,
  4.7928e-05,
  4.7928e-05,
  4.7928e-05,
  3.3131e-05,
  3.3131e-05,
  3.3131e-05,
  3.3131e-05],
 'n_rain_t0': [],
 'q_rain_t0': [],
 'n_cloud_t1': [],
 'q_cloud_t1': [],
 'n_rain_t1': [],
 'q_rain_t1': []}

In [92]:
cmom1

{'n_cloud_t0': [],
 'q_cloud_t0': [],
 'n_rain_t0': [],
 'q_rain_t0': [],
 'n_cloud_t1': [84260000.0],
 'q_cloud_t1': [],
 'n_rain_t1': [5200.5],
 'q_rain_t1': [1.6439e-05]}

In [93]:
fmom1

{'n_cloud_t0': [],
 'q_cloud_t0': [],
 'n_rain_t0': [],
 'q_rain_t0': [],
 'n_cloud_t1': [14836000.0,
  14836000.0,
  14836000.0,
  14836000.0,
  148760000.0,
  148760000.0,
  148760000.0,
  148760000.0,
  150270000.0,
  150270000.0,
  150270000.0,
  150270000.0],
 'q_cloud_t1': [7.7681e-06,
  7.7681e-06,
  7.7681e-06,
  7.7681e-06,
  4.7928e-05,
  4.7928e-05,
  4.7928e-05,
  4.7928e-05,
  3.3131e-05,
  3.3131e-05,
  3.3131e-05,
  3.3131e-05],
 'n_rain_t1': [9.0825e-06,
  9.0825e-06,
  9.0825e-06,
  9.0825e-06,
  0.00013062,
  0.00013062,
  0.00013062,
  0.00013062,
  2.9176e-05,
  2.9176e-05,
  2.9176e-05,
  2.9176e-05],
 'q_rain_t1': [2.3614e-15,
  2.3614e-15,
  2.3614e-15,
  2.3614e-15,
  3.3962e-14,
  3.3962e-14,
  3.3962e-14,
  3.3962e-14,
  7.5857e-15,
  7.5857e-15,
  7.5857e-15,
  7.5857e-15]}

In [94]:
from cffi_plugin import ffi
import numpy as np
import os
import transfer_arrays
import pytorch_lightning as pl

import sys
sys.path.append('/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/cffi_interface')
from solvers.moment_solver import simulation_forecast
sys.path.append('/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/cffi_interface/models')
sys.path.append('/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/cffi_interface')

import models.plModel as plm


In [95]:
    inputs_mean = np.asarray([[0.0002621447787797809, 51128093.51524663,
                    0.0003302890736022656, 5194.251154308974,
                    0.5566250557023539, 4.8690682855354596e-12,
                    0.0005924338523807814, 1.0848856769219835e-05,
                    2.0193905073168525]])

    inputs_std = np.asarray([[0.0003865559774857862, 86503916.13808665,
                    0.00041369562655559327, 19127.947970150628,
                    0.46107363560819126, 3.873092422358367e-12,
                    0.00042887039563850967, 1.920461805101116e-06,
                    1.3098055608321857]])

    updates_mean = np.asarray([[-8.527820407019667e-08, -13961.459867976775,
                    8.527678028525988e-08, 0.010221931180955181]])

    updates_std = np.asarray([[3.600841676033818e-07, 55095.904252313965,
                    3.6008419243808887e-07, 68.6678997504877]])

    #NN Model initalization
    pl_model = plm.LightningModel(inputs_mean=inputs_mean, inputs_std=inputs_std,
                            updates_mean=updates_mean, updates_std=updates_std) 

    hard_coded_path = '/work/ka1176/caroline/gitlab/2022-03-hereon-python-fortran-bridges/cffi_interface'
    trained_model = pl_model.load_from_checkpoint(hard_coded_path + "/trained_models/best_model.ckpt")


In [158]:
for ix in range(100):

    fortran_moments = np.array([fmom['q_cloud_t0'][ix],
                            fmom['n_cloud_t0'][ix],
                            0,
                            0])
    
    first_vals = dict(q_cloud=fmom['q_cloud_t0'][ix],
                      n_cloud=fmom['n_cloud_t0'][ix],
                      q_rain=0, n_rain=0)

    new_forecast = simulation_forecast(
            fortran_moments, trained_model,inputs_mean, inputs_std
            )
    new_forecast.test()

    [c_q_cloud_t1, c_n_cloud_t1, c_q_rain_t1, c_n_rain_t1] = new_forecast.moments_out.squeeze()

    c_direct_t1 = dict(n_cloud_t1=c_n_cloud_t1, q_cloud_t1=c_q_cloud_t1, n_rain_t1=c_n_rain_t1, q_rain_t1=c_q_rain_t1)

    lbl=['key', 'F (t0)', 'F (t1)', 'PY (t1)']
    print(f'{lbl[0]:10}  {lbl[1]}   {lbl[2]}   {lbl[3]}')
    print('-'*30)

    for kkey in ['q_cloud', 'n_cloud', 'q_rain', 'n_rain']:
        key = kkey+'_t1'        
        
        print(f'{kkey:10}: {first_vals[kkey]:.2e} {fmom[key][ix]:.2e} {c_direct_t1[key]:.2e}')
        
    print()

key         F (t0)   F (t1)   PY (t1)
------------------------------
q_cloud   : 7.77e-06 7.77e-06 0.00e+00
n_cloud   : 1.48e+07 1.48e+07 8.86e+07
q_rain    : 0.00e+00 2.36e-15 7.77e-06
n_rain    : 0.00e+00 9.08e-06 5.20e+03

key         F (t0)   F (t1)   PY (t1)
------------------------------
q_cloud   : 7.77e-06 7.77e-06 0.00e+00
n_cloud   : 1.48e+07 1.48e+07 8.86e+07
q_rain    : 0.00e+00 2.36e-15 7.77e-06
n_rain    : 0.00e+00 9.08e-06 5.20e+03

key         F (t0)   F (t1)   PY (t1)
------------------------------
q_cloud   : 7.77e-06 7.77e-06 0.00e+00
n_cloud   : 1.48e+07 1.48e+07 8.86e+07
q_rain    : 0.00e+00 2.36e-15 7.77e-06
n_rain    : 0.00e+00 9.08e-06 5.20e+03

key         F (t0)   F (t1)   PY (t1)
------------------------------
q_cloud   : 7.77e-06 7.77e-06 0.00e+00
n_cloud   : 1.48e+07 1.48e+07 8.86e+07
q_rain    : 0.00e+00 2.36e-15 7.77e-06
n_rain    : 0.00e+00 9.08e-06 5.20e+03

key         F (t0)   F (t1)   PY (t1)
------------------------------
q_cloud   : 4.79e-05 4.79e-

IndexError: list index out of range